**Crawl dữ liệu trên trang bất động sản**

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
for i in range(1, 15): 
    page_url = 'https://bdshanoi.com.vn/can-ho/next-page-%s.html' %i
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    new_feeds = soup.find('ul', class_ = 'ulpro').find_all('li')
    
    for feed in new_feeds: 
        print('\n')
        #print(feed)
        title = feed.find("div", class_ = 'thumb').find('img').get('alt')
        #print(title)
        khuvuc = feed.find("div", class_ = 'ileft').text.strip()
        #print(khuvuc)
        dientich = feed.find("div", class_ = "iright").text.strip()
        #print(dientich)
        gia = feed.find("div", class_ = 'sleft').text.strip()
        #print(gia)
        batdongsan_dataset = [(title, khuvuc, gia, dientich)]
        batdongsan_df = pd.DataFrame(batdongsan_dataset, columns = ['Title', 'Location', 'Price', 'Size'])
        csv_file = 'batdongsanfull2.csv'
        if os.path.exists(csv_file): 
            batdongsan_df.to_csv(csv_file, mode = 'a', header = False, index = None, encoding = 'utf-8')
        else: 
            batdongsan_df.to_csv(csv_file, index = None, encoding = 'utf-8')
        

**Chạy model**

In [133]:
import pandas as pd
import matplotlib.pyplot as plt

In [134]:
df = pd.read_csv('batdongsanfull.csv')

In [113]:
df.shape
df.head()

(199, 4)

In [115]:
df.dtypes

Title        object
Khu vuc      object
Gia          object
Dien tich    object
dtype: object

In [116]:
import os 
current_directory = os.getcwd()
print("Thư mục làm việc hiện tại: ", current_directory)

Thư mục làm việc hiện tại:  C:\Users\LAPTOP88


In [128]:
df['Dien tich'].value_counts()

Diện tích: 182 M2     18
Diện tích: 153 M2     16
Diện tích: 123 M2     13
Diện tích: 145 M2     11
Diện tích: 114 M2     10
                      ..
Diện tích: 155 M2      1
Diện tích: 80,1 M2     1
Diện tích: 127 M2      1
Diện tích: 95 M2       1
Diện tích: 139 M2      1
Name: Dien tich, Length: 87, dtype: int64

In [118]:
df['Gia'].value_counts()

Giá: Vui lòng liên hệ    35
Giá: 8,5 tỷ               9
Giá: 5,8 tỷ               7
Giá: 8 tỷ                 6
Giá: 9,5 tỷ               6
                         ..
Giá: 6,15 tỷ              1
Giá: 9,8 tỷ               1
Giá: 10,7 tỷ              1
Giá: 10,6 tỷ              1
Giá: 6,4 tỷ               1
Name: Gia, Length: 77, dtype: int64

In [135]:
#Loại bỏ chữ 'Khu vực:' ra khỏi trường Location bằng replace
df['Khu vuc'] = df['Khu vuc'].str.replace("Khu Vực:", "").str.strip()
df['Gia'] = df['Gia'].str.replace("Giá:", '').str.strip()
df['Dien tich'] = df['Dien tich'].str.replace("Diện tích:", '').str.strip()

#loại bỏ chữ tỷ
df['Gia'] = df['Gia'].str.replace("tỷ", '').str.strip()
df['Gia'] = df['Gia'].str.replace(",", '.').str.strip()
df['Dien tich'] = df['Dien tich'].str.replace("M2", '').str.strip()
df['Dien tich'] = df['Dien tich'].str.replace("m2", '').str.strip()
df.loc[df['Dien tich'] == '', 'Dien tich'] = '0'
#Thay dong 'vui long lien he' thành 0
df['Gia'] = df['Gia'].str.replace("Vui lòng liên hệ", '0').str.strip()
df['Dien tich'] = df['Dien tich'].str.replace(",", '.').str.strip()
df

,Title,Khu vuc,Gia,Dien tich
0,"Bán căn hộ 168m, tòa C2 Udic Westlake - Ciputra",Ciputra Hà Nội,8,168
1,"Bán căn hộ 168m, tòa C2 Udic Westlake - Ciputra",Ciputra Hà Nội,8,168
2,"Bán căn hộ 3 PN, view Hồ chung cư Water mark, ...",Quận Tây Hồ,10.5,123
3,"Bán căn hộ tòa L1 Ciputra, 153m, 2PN, 2WC view...",Ciputra Hà Nội,10.6,153
4,"Bán căn hộ tòa P1 Ciputra, 182m, 4pn, view sân...",Ciputra Hà Nội,9.7,182
...,...,...,...,...
194,"Chỉ 5,xx tỷ có ngay căn hộ 2PN view Hồ Tây thu...",Quận Tây Hồ,5.7,84
195,Bán căn hộ đẹp tầng cao view thoáng chung cư M...,Quận Đống Đa,5.2,133
196,Bán căn góc 3 phòng ngủ chung cư Sky City 88 L...,Quận Đống Đa,6.4,139
197,Bán căn hộ 2pn chung cư Vinhomes Nguyễn Chí Th...,Quận Đống Đa,6.3,86


In [130]:
df['Gia'].value_counts()

0       35
8.5      9
5.8      7
8        6
9.5      6
        ..
6.15     1
9.8      1
10.7     1
10.6     1
6.4      1
Name: Gia, Length: 77, dtype: int64

In [121]:
#hiện thị các giá trị 
# df['Khu vuc'].unique()
# df['Khu vuc'].value_counts()
# df['Gia'].value_counts()
df['Dien tich'].value_counts()

182     18
153     16
123     13
145     11
114     10
        ..
155      1
80.1     1
127      1
95       1
139      1
Name: Dien tich, Length: 87, dtype: int64

In [138]:
#Chuyen kieu gia tri
df['Gia'] = df['Gia'].astype(float)
# df['Dien tich'] = df['Dien tich'].astype(float)

In [139]:
df['Dien tich'] = df['Dien tich'].astype(float)

In [141]:
df.dtypes

Title         object
Khu vuc       object
Gia          float64
Dien tich    float64
dtype: object

In [142]:
df

,Title,Khu vuc,Gia,Dien tich
0,"Bán căn hộ 168m, tòa C2 Udic Westlake - Ciputra",Ciputra Hà Nội,8.0,168.0
1,"Bán căn hộ 168m, tòa C2 Udic Westlake - Ciputra",Ciputra Hà Nội,8.0,168.0
2,"Bán căn hộ 3 PN, view Hồ chung cư Water mark, ...",Quận Tây Hồ,10.5,123.0
3,"Bán căn hộ tòa L1 Ciputra, 153m, 2PN, 2WC view...",Ciputra Hà Nội,10.6,153.0
4,"Bán căn hộ tòa P1 Ciputra, 182m, 4pn, view sân...",Ciputra Hà Nội,9.7,182.0
...,...,...,...,...
194,"Chỉ 5,xx tỷ có ngay căn hộ 2PN view Hồ Tây thu...",Quận Tây Hồ,5.7,84.0
195,Bán căn hộ đẹp tầng cao view thoáng chung cư M...,Quận Đống Đa,5.2,133.0
196,Bán căn góc 3 phòng ngủ chung cư Sky City 88 L...,Quận Đống Đa,6.4,139.0
197,Bán căn hộ 2pn chung cư Vinhomes Nguyễn Chí Th...,Quận Đống Đa,6.3,86.0


In [143]:
#Đổi tên biến (attributes)
df.rename(columns={'Khu vuc': 'Location'}, inplace=True)
df.rename(columns={'Gia': 'Price'}, inplace=True)
df.rename(columns={'Dien tich': 'Size'}, inplace=True)